In [11]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, StepSequence
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails

ws = Workspace.from_config()
blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'mdrift_pipeline'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = "vdsm-cluster-d13"  # ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute = AmlCompute(ws, aml_compute_target)

# Use an existing env which was registered to the ws
env = Environment.get(workspace=ws,name="vdsm_all",version="5")

In [12]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

script_run_config = RunConfiguration()
script_run_config.target = aml_compute
script_run_config.environment = env

veg_type_pipeline_param = PipelineParameter(name="veg_type", default_value='bs')
end_month_pipeline_param = PipelineParameter(name="end_month", default_value='2022-01')

example_step = PythonScriptStep(
    name="mdrift_pipeline",
    script_name="drift_pipeline_dev.py",
    arguments=[
        "--veg_type", veg_type_pipeline_param,
        "--end_month", end_month_pipeline_param
    ],
    runconfig=script_run_config,
    source_directory="."
    )

step_sequence = StepSequence(steps=[example_step])

pipeline = Pipeline(workspace=ws, steps=step_sequence)
# pipeline_run = exp.submit(pipeline)
# print('Datadrift pipeline submitted for execution')
# RunDetails(pipeline_run).show()
# pipeline_run.wait_for_completion()

In [13]:
pipeline.publish(name="data_drift", description="Detect data drift", version=2)

Created step mdrift_pipeline [2884b1a8][85750496-abcf-4c8c-ab05-8a6a1131c887], (This step will run and generate new outputs)


Pipeline(Name: data_drift,
Id: af303441-6626-4661-8b42-849d1b426061,
Status: Active,
Endpoint: https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourceGroups/arg-rt-pacedev-spacetech/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-spacetech001/PipelineRuns/PipelineSubmit/af303441-6626-4661-8b42-849d1b426061)